<a href="https://colab.research.google.com/github/MocT117/Another-one-/blob/master/Sem%C3%A1foro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go

st.title("Dashboard Semáforo de Fechas PO vs EXW Real")
st.write("Carga tu archivo Excel para visualizar el semáforo de fechas:")

# Subida de archivo desde el dashboard (NO necesitas ruta manual, ni Colab)
archivo = st.file_uploader("Sube tu archivo Excel", type=["xlsx"])

if archivo:
    df = pd.read_excel(archivo)

    # Convertir a fecha
    df['Fecha de PO'] = pd.to_datetime(df['Fecha de PO'], errors='coerce')
    df['EXW Real'] = pd.to_datetime(df['EXW Real'], errors='coerce')

    # Calcular diferencia de días
    df['Diferencia días calculada'] = (df['EXW Real'] - df['Fecha de PO']).dt.days

    # Semáforo
    def status_semáforo(dias):
        if pd.isna(dias):
            return 'Sin fecha'
        elif 0 <= dias <= 6:
            return 'En tiempo'
        elif dias >= 7:
            return 'On Delay'
        else:
            return 'Revisar'

    df['Semáforo'] = df['Diferencia días calculada'].apply(status_semáforo)

    # Mostrar tabla con semáforo
    st.subheader("Tabla con Semáforo")
    st.dataframe(df[['Fecha de PO', 'EXW Real', 'Diferencia días calculada', 'Semáforo']])

    # Gauge chart con Plotly
    st.subheader("Indicador Gauge: % En Tiempo")

    total_validos = df['Semáforo'].isin(['En tiempo', 'On Delay']).sum()
    en_tiempo = (df['Semáforo'] == 'En tiempo').sum()

    if total_validos > 0:
        porcentaje_en_tiempo = en_tiempo / total_validos * 100
    else:
        porcentaje_en_tiempo = 0

    fig = go.Figure(go.Indicator(
        mode = "gauge+number+delta",
        value = porcentaje_en_tiempo,
        title = {'text': "% En Tiempo (0-6 días)"},
        gauge = {
            'axis': {'range': [None, 100]},
            'steps': [
                {'range': [0, 60], 'color': "#F5B7B1"},    # rojo
                {'range': [60, 85], 'color': "#F9E79F"},   # amarillo
                {'range': [85, 100], 'color': "#ABEBC6"}   # verde
            ],
            'threshold': {
                'line': {'color': "black", 'width': 4},
                'thickness': 0.75,
                'value': porcentaje_en_tiempo
            }
        }
    ))

    st.plotly_chart(fig)

    st.info("Dashboard generado por Kai 🤖")

else:
    st.warning("Sube un archivo para ver el dashboard.")
